In [ ]:
# from multiprocessing import Pool, Manager
from tqdm import tqdm

class NgramModel():
    def __init__(self):
        pass
    
    def load_data(self, train_data_file):
        self.train_data = []
        with open(train_data_file, 'r') as fin:
            for line in fin:
                self.train_data.append(["<s>"] + line.strip().split(" ") + ["</s>"])
        
                
    def unsmoothed_n_grams(self, n, num_thread=4):
        word_count = {}
        
        def ngrams(lst_words, n):
            if len(lst_words) < n:
                return 
            for i in range(len(lst_words) - n + 1):
                key = " ".join(lst_words[i:i+n])
                word_count.setdefault(key, 0)
                word_count[key] += 1
                
        for lst_words in tqdm(self.train_data):
            ngrams(lst_words, n)
        
        return word_count

In [ ]:
model = NgramModel()

model.load_data("../data/comment_corpus_for_lm.train.txt")

unigram = model.unsmoothed_n_grams(1)
bigram = model.unsmoothed_n_grams(2)
trigram = model.unsmoothed_n_grams(3)

In [ ]:
import cPickle as pkl

pkl.dump((unigram, bigram, trigram), open("./DQD_vocab.pkl", 'wb'))